## 1. Ejemplo de clasificación con KNN
#Identificación de Sargazo

Infirmación tomada del artículo ERISNet: deep neural network for
Sargassum detection along the coastline
of the Mexican Caribbean
https://peerj.com/preprints/27445v1/#supp-1


Número de instancias: 4515

Atributos : 14

Output: Sargazo o No sargazo

#2. Importación de librerías y montando google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import _____ as np
import _____ as pd
import matplotlib.pyplot as _____
import seaborn as sns
import itertools
import os

In [ ]:
from sklearn.preprocessing import ______ #Codificación de etiquetas
from sklearn.preprocessing import ______, ______ #Escaladores
from sklearn.model_selection import ______ #holdout validation
from sklearn.metrics import confusion_matrix, accuracy_score 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
path = r'/content/drive/Shareddrives/Data Science para Geociencias/6. Métodos de ML/6.6 K-vecinos cercanos'
dataset_path = os.path.join(path,'sargazo.csv')

# 3. Lectura y visualuzación del dataset

In [ ]:
train_df = ___________(dataset_path) 
clases = train_df['label'].unique()
train_df.dropna(axis=0, inplace=True)
train_df

In [ ]:
plt.figure(figsize=(15, 9))
sns.______('label',data=train_df) #contar instancias por "label"
plt.title('Distribución de Clases')
plt.xlabel('Class')
plt.ylabel('Counts')

plt.show()

In [ ]:
train_df.describe()

#4. Escalamiento

In [ ]:
scaler = ______() 
train_df.loc[:, train_df.columns != 'label'] = scaler.fit_transform(train_df.loc[:, train_df.columns != 'label'])

#5. Codificación de etiquetas

In [ ]:
le = LabelEncoder()
train_df['label'] = le.______(train_df.label.values)
train_df

#6.Detección de outliers

In [ ]:
lof = LocalOutlierFactor(n_neighbors=20, metric='euclidean')
yhat = lof.fit_predict(train_df)
# select all rows that are not outliers
mask = yhat != -1
train_df= train_df.loc[mask, :]


print('Número de outliers detectados', (~mask).sum())
train_df

#7. Dividiendo el dataset en entrenamiento y prueba

In [ ]:
X = train_df.iloc[:, :-1].values
y = train_df.iloc[:, -1].values

In [ ]:
X_train, X_test, y_train, y_test = ______(X, y, test_size = ______, random_state = 0, stratify=y)

print(f'Dimension de los datos de entrenamiento {X_train.shape}')
print(f'Dimension de los datos de prueba {X_test.shape}')

#8. Selección del valor de k

In [ ]:
k_n = []
score_train = []
score_test = []

for i in range(15):
  classifier = KNeighborsClassifier(n_neighbors = i+1, metric = 'euclidean', p = 2)
  classifier.fit(X_train, y_train) #entrena
  y_pred = classifier.predict(X_test) #prueba
  score_t = classifier.score(X_train, y_train) #score train 
  score = accuracy_score(y_test, y_pred) #score test
  k_n.append(i+1)
  score_train.append(score_t)
  score_test.append(score)

plt.figure(figsize=(15, 9))
plt.plot(k_n,score_train, color = 'red')
plt.plot(k_n,score_test, color = 'green')
plt.title('Relación número de vecinos vs accuracy')
plt.legend(['Entrenaminto', 'Prueba'])
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.grid(True)
plt.xlim(np.min(k_n), np.max(k_n))
plt.ylim(0.5, 1)


#9. Entrenando el modelo de KNN con el valor de K óptimo

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = ______, metric = '______', p = 2)

KNN.fit(X_train, y_train)
KNN.score(X_train, y_train)

#10. Prediciendo resultados

In [ ]:
y_pred = KNN.predict(______)
score = accuracy_score(y_test, y_pred)
print(score)

In [ ]:
#Funciones para MC y métricas
def plot_confusion_matrix(cm, classes, tit, normalize=False):
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)
        title, fmt = 'Matriz de confusión normalizada', '.2f'
    else:
        title, fmt = tit, 'd'
    plt.figure(figsize=(10,8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)#, fontsize=12)
    plt.colorbar(pad=0.05)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=40)
    plt.yticks(tick_marks, classes)
    thresh = cm.max()/2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),horizontalalignment="center", 
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Clase Verdadera')#, fontsize=10)
    plt.xlabel('Clase Predicha')#, fontsize=10)
    plt.savefig(title+'.png')
    #plt.grid(b=None)
    plt.show()
def sens_spec(cls_names, y_true, y_pred, pesos):
  sensitivity = []
  specificity = []
  prec=[]
  fscore = []
  sd = []
  for i,name in enumerate(cls_names):
    TP = np.sum((y_true==name) & (y_pred==name))
    TN = np.sum((y_true!=name) & (y_pred!=name))
    FP = np.sum((y_true!=name) & (y_pred==name))
    FN = np.sum((y_true==name) & (y_pred!=name))
    sensitivity.append(TP/(TP+FN))
    specificity.append(FP/(TN+FP))
    prec.append(TP/(TP+FP))
    fscore.append(2*(prec[i]*sensitivity[i])/(prec[i]+sensitivity[i]))
  sensitivity.append(sum([x*y for x,y in zip(sensitivity,pesos)]))
  specificity.append(sum([x*y for x,y in zip(specificity,pesos)]))
  prec.append(sum([x*y for x,y in zip(prec,pesos)]))
  fscore.append(sum([x*y for x,y in zip(fscore,pesos)]))
  for i in range(len(cls_names)):
    sd.append(np.sqrt((sensitivity[i]-sensitivity[-1])**2))
  sd.append(np.nan)
  return sensitivity, specificity, prec, fscore, sd

Matriz de confusión

In [ ]:
predictions = np.float32(y_pred)
true_labels = np.float32(y_test)
n_classes = 2
cnf_matrix = confusion_matrix(true_labels, predictions, labels=range(n_classes))
tit = 'Matriz de confusión con KNN'
plot_confusion_matrix(cnf_matrix,clases, tit, normalize=False)

In [ ]:
pesos = train_df['label'].value_counts().sort_index().tolist()/ np.sum(train_df['label'].value_counts().tolist())
print(pesos)

Métricas por clase 

In [ ]:
sensitivity, FPR, prec, fscore, sd = sens_spec(range(n_classes), true_labels, predictions, pesos)
d = {'TPR':sensitivity, 'FPR':FPR, 'Acc':prec, 'F-score':fscore}
ind = clases.tolist()+['Promedio']
df = pd.DataFrame(d, index=ind)#.transpose()
index = df.index
precprom = sum([x*y for x,y in zip(prec,pesos)])
index.name = 'Acc Pond: %s'%precprom
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(df, annot=True)